# [WIP] Spin echo simulation

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [150]:
def new_range(img : np.ndarray, min = 0.0, max = 1.0):
  '''Function that converts an array to the given desired range.'''
  return ((img - np.min(img))/(np.max(img) - np.min(img)))*(max - min) + min


def normalize(vec : np.ndarray) -> np.ndarray:
  """Normalizes an array or a set of arrays.

  Parameters
  ----------
  vec : np.ndarray (N, M)
    Array to be normalized."""
  return vec/np.tile(np.linalg.norm(vec, axis = 0), vec.shape[0]).reshape(vec.shape)


def rotate(vec : np.ndarray, alpha : float, beta : float, gamma : float):
    """Rotates a 3D array or set of arrays by a certain set of angles,in radians. 
    The order of rotation is Rxy*Rxz*Ryz, so it first rotates around 
    the x axis, then around the y axis, then around the z axis.

    The matrix of rotations are:

    Rxy = np.matrix([[ np.cos(alpha), np.sin(alpha), 0.0], 
                     [-np.sin(alpha), np.cos(alpha), 0.0], 
                     [           0.0,           0.0, 1.0]])
    
    Rxz = np.matrix([[  np.cos(beta),           0.0,           np.sin(beta)], 
                     [         0.0,            1.0,                   0.0], 
                     [ -np.sin(beta),           0.0,           np.cos(beta)]])
    
    Ryz = np.matrix([[ 1.0,              0.0,             0.0], 
                     [ 0.0,    np.cos(gamma),   np.sin(gamma)], 
                     [ 0.0,   -np.sin(gamma),   np.cos(gamma)]])
    
    Parameters
    ----------
    vec : np.ndarray (N, 3)
      Array to be rotated.
    alpha : float
      angle of rotation around the z axis.
    beta : float
      angle of rotation around the y axis.
    gamma : float
      angle of rotation around the x axis.
      
    """

    
    R = np.matrix([[np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma) - np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma) + np.sin(alpha)*np.sin(gamma)], 
                   [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma) + np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma) - np.cos(alpha)*np.sin(gamma)], 
                   [-np.sin(beta), np.cos(beta)*np.sin(gamma), np.cos(beta)*np.cos(gamma)]])
    
    return np.asarray((R*np.matrix(vec).T).T)


def apply_B(M : np.ndarray, B : np.ndarray, gamma : float, dt : float):
  """Applies a magnetic field B onto a magnetic moment M in a 
     certain interval dt, making it rotate about an angle w*dt, 
     with w = gamma*B_z, gamma being the gyromagnetic ratio of 
     the nucleus, and B_z the component of the magnetic field 
     orthogonal to the magnetic moment.
     
     Parameters
     ----------
     
     M : np.ndarray
      The magnetic moment of the atom
     B : np.ndarray
      The magnetic field to be applied.
     gamma : float
      The gyromagnetic moment of the nucleus
     dt : float
      The value of the simulation's dt"""

  x = normalize(M)
  y = np.cross(x, normalize(B))
  z = np.cross(x, y)

  B_z = np.dot(B, z)

  w = gamma*B_z
  f = w/(2.0*np.pi)

  angle = f*dt

  #first, a new coordinate system is defined 
  #so the rotations can be done in terms of angle
  new_coord = np.matrix([x,
                         y,
                         z]).T
  
  #it is also important to understand how do the original coordinate
  #system changes in the new one, so we can use it to come back
  
  #the vector is translated into the new coordinate system
  new_vec = np.array(new_coord*(np.matrix(M).T)).T[0]

  #the rotation is done
  rotated = rotate(new_vec, -angle, 0.0, 0.0)

  #we come back to the old coordinate system now with a rotated vector
  back = np.array((new_coord*np.matrix(rotated).T)).T[0]
  
  return back



In [100]:
n = 3

u = 1.0

spins = np.random.rand(n, 3)

In [141]:
v = np.array([0.0, 1.0, 0.0])

vl = rotate(v, 0.0, 0.0, np.pi/2)

print(v)
print(vl)


[0. 1. 0.]
[[0.000000e+00 6.123234e-17 1.000000e+00]]


In [160]:
u = np.array([0.0, 0.0, 1.0])
v = np.array([0.0, 1.0, 0.0])
w = np.array([1.0, 0.0, 0.0])

mat = np.matrix([u, 
                 v, 
                 w]).T

print(mat)


vec = np.array([1.0, 0.0, 2.0])

changed = np.array((mat*np.matrix(vec).T)).T[0]

print((mat*np.matrix(changed).T).T)






[ 0.  0. -1.]
